This notebook holds the code to perform the steps needed for Portal migration, starting with the loading of the correct libraries. This notebook relies on the samples provided by ESRI here: https://github.com/Esri/arcgis-python-api

Note that the environment.yml file is very old and won't build unless you remove the version locks on the packages in it

In [ ]:
import os, csv, datetime
from arcgis.gis import *
import creds

# portal_url = "https://gis.brwncald.com/portal"
# portal_name = "oldportal"
# portal_url = "https://azr-gisprd02.bc.brwncald.com/portal"
portal_url = "https://gis.brwncald.com/portal"
portal_name = "newportal"

bcgis = GIS(portal_url, creds.username, creds.password, verify_cert=False)
display(bcgis)

Dump out a listing of groups, users, and content to CSVs

In [ ]:
f = open(portal_name + ".groups.csv", 'w', newline='', encoding='utf-8')
c = open(portal_name + ".grp_content.csv", 'w', newline='', encoding='utf-8')
u = open(portal_name + ".grp_users.csv", 'w', newline='', encoding='utf-8')
groups = []

fieldnames = ['id',
 'title',
 'isInvitationOnly',
 'owner',
 'admins',
 'users',
 'description',
 'snippet',
 'tags',
 'phone',
 'sortField',
 'sortOrder',
 'isViewOnly',
 'thumbnail',
 'created',
 'modified',
 'created_UTC',
 'modified_UTC',
 'access',
 'capabilities',
 'isFav',
 'isReadOnly',
 'protected',
 'autoJoin',
 'notificationsEnabled',
 'provider',
 'providerGroupName',
 'leavingDisallowed',
 'hiddenMembers',
 'displaySettings',
 'membershipAccess',
 '_gis',
 '_migrate',
 '_portal',
 'groupid',
 '_workdir',
 '_hydrated',
 'featuredItemsId', 'orgId', 'properties', 'typeKeywords']
writer = csv.DictWriter(f, fieldnames, quoting=csv.QUOTE_ALL)
writer.writeheader()
content = csv.DictWriter(c, ['groupID', 'itemID'], quoting=csv.QUOTE_ALL)
content.writeheader()
users = csv.DictWriter(u, ['groupID', 'membertype', 'username'], quoting=csv.QUOTE_ALL)
users.writeheader()

cnt = 0
for group in bcgis.groups.search():
    data = {key: getattr(group, key) for key in list(group.keys()) if not key.startswith('_') }
    data['created_UTC'] = datetime.datetime.fromtimestamp(data['created']/1000.0)
    data['modified_UTC'] = datetime.datetime.fromtimestamp(data['modified']/1000.0)
    all_users = group.get_members()
    all_content = group.search('1=1')
    all_content2 = group.content()
    
    data.update(all_users)
    users.writerow({'groupID': group.id, 'membertype': 'owner', 'username': all_users['owner']})
    for j in all_users['admins']:
        users.writerow({'groupID': group.id, 'membertype': 'admin', 'username': j})
    for j in all_users['users']:
        users.writerow({'groupID': group.id, 'membertype': 'user', 'username': j})
    for i in group.content():
        content.writerow({'groupID': group.id, 'itemID': i.id})
    writer.writerow(data)
    groups.append(data)
    cnt+=1

f.close()
c.close()
u.close()
display(f'Found {cnt} groups')

In [ ]:
f = open(portal_name + ".users.csv", 'w', newline='', encoding='utf-8')
fieldnames = ['username', 'id', 'fullName', 'firstName', 'lastName', 'preferredView', 'description', 
              'email', 'userType', 'idpUsername', 'favGroupId', 'lastLogin', 'mfaEnabled', 
              'validateUserProfile', 'access', 'storageUsage', 'storageQuota', 'orgId', 'level', 
              'userLicenseTypeId', 'disabled', 'tags', 'culture', 'cultureFormat', 'region', 'units',
                'thumbnail', 'created', 'modified', 'provider', 'roleId', 'role', 'created_UTC', 
                'modified_UTC', 'content', 'emailStatus', 'mfaEnforcementExempt', 'udn', 'categories']
writer = csv.DictWriter(f, fieldnames, quoting=csv.QUOTE_ALL)
writer.writeheader()
cnt = 0
ids = []
users = []
for user in bcgis.users.search(max_users=600):
    data = {key: getattr(user, key) for key in list(user.keys()) if not key.startswith('_') }
    data['created_UTC'] = datetime.datetime.fromtimestamp(data['created']/1000.0)
    data['modified_UTC'] = datetime.datetime.fromtimestamp(data['modified']/1000.0)

    items = bcgis.content.advanced_search(f'owner: {user.username}', max_items=600)
    for item in items['results']:
            ids.append(item.id)

    writer.writerow(data)
    users.append(data)
    cnt+=1

f.close()
display(f'Found {cnt} users')


In [ ]:
display(users[1])

In [ ]:
f = open(portal_name + ".items.csv", 'w', newline='', encoding='utf-8')
fieldnames = ['id', 'owner', 'created', 'isOrgItem', 'modified', 'guid', 'orgId',
              'name', 'title', 'type', 'typeKeywords', 'origin', 'sourceUrl',
              'description', 'tags', 'snippet', 'thumbnail', 'documentation', 'extent', 'categories', 
              'spatialReference', 'accessInformation', 'licenseInfo', 'culture', 'properties', 'url', 
              'proxyFilter', 'access', 'appCategories', 'industries', 'languages', 'largeThumbnail', 
              'banner', 'screenshots', 'listed', 'listingProperties', 'listingPublishedDate', 'listingApproved', 
              'ownerFolder', 'protected', 'numComments', 'numRatings', 'avgRating', 'numViews', 'scoreCompleteness',
                'groupDesignations', 'created_UTC', 'modified_UTC', 'tables', 'layers', 'size', 'commentsEnabled', 
                'itemControl', 'privateUrl', 'contentStatus', 'lastViewed', 'apiToken1ExpirationDate', 
                'apiToken2ExpirationDate', 'advancedSettings', 'subInfo']
writer = csv.DictWriter(f, fieldnames, quoting=csv.QUOTE_ALL)
writer.writeheader()
cnt = 0
for id in ids:
    item = bcgis.content.get(id)
    if not item:
       print("Skipping", id)
       continue
    data = {key: getattr(item, key) for key in list(item.keys()) if not key.startswith('_') }
    data['created_UTC'] = datetime.datetime.fromtimestamp(data['created']/1000.0)
    data['modified_UTC'] = datetime.datetime.fromtimestamp(data['modified']/1000.0)
    for fld in ['description', 'snippet', 'accessInformation', 'licenseInfo']:
      if data[fld]: data[fld] = data[fld].replace('\r', ' ').replace('\n', ' ')
    
    # display(data)
    writer.writerow(data)
    cnt+=1

f.close()
display(f'Found {cnt} items')

Create Enterprise users (code copied from the CloneUsersGroupsSharingshell.py sample in samples\03_org_administrators)

In [ ]:
users = [
     {
          'username': 'jdoe@brwncald.com',
 'fullName': 'John Doe',
 'firstName': 'John',
 'lastName': 'Doe',
 'preferredView': None,
 'description': 'TEST',
 'email': 'jdoe@brwncald.com',
 'userType': 'arcgisonly',
 'idpUsername': 'jdoe@brwncald.com',
 'favGroupId': '4301122c19b04102bc570f2697518dc9',
 'validateUserProfile': True,
 'access': 'public',
 'level': '2',
 'userLicenseTypeId': 'creatorUT',
 'disabled': False,
 'tags': [],
 'culture': 'en-US',
 'cultureFormat': 'us',
 'region': None,
 'units': 'english',
 'thumbnail': None,
 'provider': 'enterprise',
 'roleId': 'org_publisher',
 'role': 'org_publisher',
 'groups': []
       }
]

In [ ]:
users = []

with open('oldportal.users.csv', 'r', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        if not row['username'].startswith("_"):
            users.append(row)

active_users = []
with open('active_bcers.csv', 'r', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        active_users.append(row['UserName'])

display(users[13])
display(active_users[13])


In [ ]:



def copy_user(target, user):

    testuser = target.users.get(user['username'])
    print(user['username'])
    if not testuser is None:
        # print("\tUsername {} already exists in Target Portal; skipping user creation".format(user['username']))
        target_user = testuser
    else:
        if user['username'].lower() in active_users:
            
            if 'role' in user and user['role'] in ['Viewer', 'User', 'Data Editor', 'Publisher', 'Administrator']:
                target_user = target.users.create(username=user['username'], password='None', firstname=user['firstName'], lastname=user['lastName'], email=user['email'], 
                    role=user['role'], description=user['description'], provider='enterprise', idp_username=user['idpUsername'], user_type=user['userLicenseTypeId'])

                print("\tUser {} was created".format(user['username']))
            else:
                print("\tUnknown role '{}' for {}".format(user['role'], user['username']))
                
        # else:
        #     print("\tUsername {} is not an active BCer; skipping user creation".format(user['username']))


for user in users:
    copy_user(bcgis, user)
    

In [ ]:
groups = []

with open('oldportal.groups.csv', 'r', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        groups.append(row)

for old_group in groups:
    # display(old_group)
    group_title = old_group['title']
    new_group_lst = bcgis.groups.search(f'title:"{group_title}"')
    if new_group_lst: #exists
        new_group = new_group_lst[0]
        users_to_add = []
        existing_users = old_group['users'].replace('"','').replace("'",'').replace(', ',',').split(',')
        for user in existing_users:
            new_portal_user = bcgis.users.get(user)
            if new_portal_user and user not in new_group.get_members():
                users_to_add.append(new_portal_user.username)
        if users_to_add:
            new_group.add_users(users_to_add)
            print("\tAdded {} to group {}".format(users_to_add, group_title))